In [1]:
#!pip install n2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 23.0 MB/s 
  Created wheel for n2: filename=n2-0.1.7-cp37-cp37m-linux_x86_64.whl size=2303034 sha256=bbb4851869012dc4bbeeedd898c9b8da3f9e5b5b869c6d83668bd9f8c8960ab5
  Stored in directory: /root/.cache/pip/wheels/36/da/12/b157ca1c9dcdd5fd3fa5e15b7823f805396fb6e6b30427465a
Successfully built n2


In [2]:
from n2 import HnswIndex
import pandas as pd

In [ ]:
music_feature = pd.read_csv('__file_location__/music_data_220527.csv', index_col = None)

In [ ]:
#'happiness' : 0, 'angry' : 1, 'sadness' : 2, 'disgust' :3 , 'surprise' : 4,'fear' :5
selected_music = music_feature[(music_feature['labels'] == 'Happy') | (music_feature['labels'] =='Angry') | (music_feature['labels'] =='Sad') | (music_feature['labels'] =='Calm') | (music_feature['labels'] =='Dramatic') | (music_feature['labels'] =='Dark')]
selected_music.reset_index(inplace=True, drop = True)

In [ ]:
music_index = HnswIndex(57)
for i in range(len(selected_music)):
    music_index.add_data(selected_music.iloc[i, 3:])
music_index.build()

In [ ]:
music_index.save('six_genre_index.hnsw')

In [ ]:
selected_music.head()

,Unnamed: 0,music_names,labels,chroma_rms_mean,chroma_rms_var,chroma_stft_mean,chroma_stft_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,...,mfcc_mean_16,mfcc_var_16,mfcc_mean_17,mfcc_var_17,mfcc_mean_18,mfcc_var_18,mfcc_mean_19,mfcc_var_19,mfcc_mean_20,mfcc_var_20
0,0,Tell Me That I Can't (Clean) - NEFFEX.mp3,Angry,0.336170,0.030709,0.504131,0.084681,2118.428100,9.756422e+05,2458.834163,...,4.136426,44.906210,-0.065393,43.347076,2.655090,49.822586,-3.587426,39.159620,-0.362364,44.010452
1,1,It's Only Worth It If You Work For It (Clean) ...,Angry,0.229996,0.011787,0.382390,0.096317,1986.543565,9.884038e+05,2333.305230,...,-1.104181,53.218056,-9.090735,58.107212,-4.072828,67.380030,-9.011088,55.313988,-6.383667,83.276054
2,2,Statement (Clean) - NEFFEX.mp3,Angry,0.254628,0.014235,0.435461,0.086562,2186.280481,7.754239e+05,2490.128772,...,1.895623,31.033130,-6.536465,38.985590,0.901428,35.720013,-3.552503,30.856724,-3.057429,32.160133
3,3,No Turning Back (Clean) - NEFFEX.mp3,Angry,0.194401,0.017006,0.357778,0.104193,1768.157461,1.897606e+06,2050.120820,...,-0.417216,66.672590,-10.211936,80.495500,-5.495617,131.986180,-7.060772,68.881490,-1.843115,63.655922
4,4,No Turning Back (Instrumental) - NEFFEX.mp3,Angry,0.172122,0.019367,0.321783,0.111694,795.101779,3.382726e+05,1067.885933,...,-1.446617,66.532300,-8.476983,83.030464,-10.983328,101.891640,-3.865174,68.955270,-2.467369,56.117060


In [ ]:
# 0~119 : Angry | 120~239 : Happy | 240~359 : Calm | 360~479 : Dark | 480~599 : Sad | 600~719 : Dramatic

ID 넣으면 음악 추천해주는 Class

In [11]:
!pip install n2
from n2 import HnswIndex
import pandas as pd

class music_recommendation():
  def __init__(self):
    self.m_index = HnswIndex(57)
    # 저장된 인덱스 호출
    self.m_index.load('__file_location__/six_genre_index.hnsw')
    
    # 음악 추출 데이터 호출
    music_feature = pd.read_csv('__file_location__/music_data_220527.csv', index_col = None)
    
    # 6가지 감정만 사용
    selected_music = music_feature[(music_feature['labels'] == 'Happy') | (music_feature['labels'] =='Angry') | (music_feature['labels'] =='Sad') | (music_feature['labels'] =='Calm') | (music_feature['labels'] =='Dramatic') | (music_feature['labels'] =='Dark')]
    selected_music.reset_index(inplace=True, drop = True)

    self.index_and_music = {i:k for i, k in enumerate(selected_music['music_names'])}

  def recommendation_by_id(self, id):
    # 0~119 : Angry
    if id in range(0,120):
      music_mood = 'Angry'
      music_range = range(0,120)
      music_list = [musics for musics in self.m_index.search_by_id(id, 720) if musics in music_range]
    # 120~239 : Happy
    elif id in range(120,240):
      music_mood = 'Happy'
      music_range = range(120,240)
      music_list = [musics for musics in self.m_index.search_by_id(id, 720) if musics in music_range]
    # 240~359 : Calm
    elif id in range(240,360):
      music_mood = 'Calm' 
      music_range = range(240,360)
      music_list = [musics for musics in self.m_index.search_by_id(id, 720) if musics in music_range]
    # 360~479 : Dark
    elif id in range(360,480):
      music_mood = 'Dark'       
      music_range = range(360,480)
      music_list = [musics for musics in self.m_index.search_by_id(id, 720) if musics in music_range]
    # 480~599 : Sad
    elif id in range(480,600):
      music_mood = 'Sad'         
      music_range = range(480,600)
      music_list = [musics for musics in self.m_index.search_by_id(id, 720) if musics in music_range]
    # 600~719 : Dramatic
    elif id in range(600,720):
      music_mood = 'Dramatic'   
      music_range = range(600,720)
      music_list = [musics for musics in self.m_index.search_by_id(id, 720) if musics in music_range]
    
    return music_mood, [self.index_and_music[recomend_music] for recomend_music in music_list[1:21]]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
test = music_recommendation()

In [14]:
atmosphere, music_list_test = test.recommendation_by_id(563)
print(atmosphere, music_list_test)

Sad ['Somewhere Fuse - French Fuse.mp3', 'Boards - Francis Preve.mp3', "Sad Sunday - E's Jammy Jams.mp3", 'Scapes - Silent Partner.mp3', 'Mist - Odonis Odonis.mp3', 'Coming Home - Dan Lebowitz.mp3', 'Through and Through - Amulets.mp3', 'Surrender - Dan Lebowitz.mp3', 'Last Train to Mars - Dan Lebowitz.mp3', 'Called Upon - Silent Partner.mp3', "80's Video Game Death - Sir Cubworth.mp3", 'Ether - Silent Partner.mp3', 'Sailing - Anno Domini Beats.mp3', 'Bring Me Your Sorrows - Dan Lebowitz.mp3', 'Late Night Train - Vans in Japan.mp3', 'Back Home - Coyote Hearing.mp3', 'Till I Let Go - NEFFEX.mp3', 'Frost - HOVATOFF.mp3', 'Elegy - Asher Fulero.mp3', "Suzuki's Theme - Endless Love.mp3"]
